 # 1. Building some simple model

- We will build a simple model whuch uses the similarity measures.


- It is ___ might be worse than___ any of the basic prediction algorithms. (__like, THE BASELINE MODEL__, with biases)


- It is just to show that, How we can use __similarity matrices__ while recommending OR predicting the user's rating/interest in some perticular movie/item.

### 1.1 Using MOVIE_MOVIE similarity

__Algorithm__:

 $\Large \hat r_{ui} = \frac{\text { Global Average Rating } + \frac {\sum (\text{User Ratings of similar movies})} {\text {No of ratings }}} {2} $

- We will take __top 10__ ratings of similar movies by this user, and take average of them.


- We will take this ( avg rating of similar movies by this user) and Global average of ratings, We again average them.


- It is not the best model, but, it is not that bad.

___why top 10..? why not top 20..??___

- It is just for this model. Later we will build __a Regression out of this top_10 or top_20 features__ and let the model figure it out which ratings to consider.


- We will also add some other features to this regression problem. like _user average rating_, _movie average rating_, _global average rating_, ...., ..etc.,. 

__ get the SImilarity_matrix from the disk__

In [5]:
# let's read movie_movie similarity matrix from the disk if if it is present
import os
from scipy import sparse

if os.path.isfile('m_m_sim_sparse.npz'):
    # just read from the disk 
    print('reading it from disk..')
    m_m_sim_sparse = sparse.load_npz('m_m_sim_sparse.npz')
    print('Our similarity matrix is of',m_m_sim_sparse.shape,' size')
else:
    # We dont't want to create one. So,
    print("Oops..!, you don't have the m_m_similarity file in your directory. Get it first..")

Our similarity matrix is of (17771, 17771)  size


In [13]:
def average_of_top_similar_movies(similarities, ratings, top=100, verbose=False):
    # get all similarities into an numpy array.
    sims = similarities.toarray().ravel() if sparse.issparse(similarities) else similarities
    # get the top similar users or items, we don't care about those values...
    top_movies = np.argsort(sims)[::-1][1:top+1] # we are ignoring the movie/user itself..
    
    top_ratings = ratings.toarray().ravel() if sparse.issparse(ratings) else ratings
    if verbose:
        print(sum(top_ratings[top_users_or_movies] != 0))
        print(sum(top_ratings[top_users_or_movies]))
        print(len(top_users_or_movies))
    # return the average rating of top ''top'' users or movies
    return sum(top_ratings[top_users_or_movies]) / (sum(top_ratings[top_users_or_movies]!=0))

__ get the TEST data  from the disk__

In [1]:
import pandas as pd
import numpy as np

In [8]:
if os.path.isfile('test.csv'):
    print('reading it from the file')
    test_df = pd.read_csv('test.csv', usecols=['movie', 'user', 'rating'])
    print("Done..")
else:
    print("test.csv is not present in the present working direcory.")

reading it from the file
Done..


In [11]:
test_df.head()

,movie,user,rating
0,4149,1706831,3.0
1,6166,1821144,3.0
2,3917,1928592,3.0
3,1798,2256821,3.0
4,12911,94782,3.0


In [12]:
sum(test_df.duplicated())

0